In [1]:
pip install nvcc4jupyter

In [2]:
%load_ext nvcc4jupyter

Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmprwbppn3b".


In [3]:
!apt-get update && apt-get install cuda-toolkit-12-4

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://cli.github.com/packages stable InRelease [3,917 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,006 kB]
Hit:7 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 https://cli.github.com/packages stable/main amd64 Packages [346 B]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,264 kB]
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/pp

In [4]:
!export PATH="/usr/local/cuda-12.4/bin:$PATH"
!export LD_LIBRARY_PATH="/usr/local/cuda-12.4/lib64:${LD_LIBRARY_PATH}"

In [5]:
!ln -sfn /usr/local/cuda-12.4 /usr/local/cuda

In [6]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Mar_28_02:18:24_PDT_2024
Cuda compilation tools, release 12.4, V12.4.131
Build cuda_12.4.r12.4/compiler.34097967_0


In [7]:
!nvidia-smi

Mon Sep 15 01:36:59 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   41C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [8]:
!mkdir /home/include

In [9]:
%%writefile /home/include/utils.h
#ifndef UTILS_H
#define UTILS_H

#include <iostream>
#include <omp.h>

void fillRandom(float* array, int totalDim){
#pragma omp parallel for
  for (int i = 0; i < totalDim; i++) {
    array[i] = rand() % 10;
  }
}

void initialize(float * array, int totalDim){
#pragma omp parallel for
  for (int i = 0; i < totalDim; i++) {
    array[i] = 0;
  }
}


class Ref{
  public:
    static const int M = 4;
    static const int N = 2;
    static const int K = 3;

    float A[M*K] = {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12};
    float B[K*N] = {1, 2, 3, 4, 5, 6};
    float C[M*N] = {22, 28, 49, 64, 76, 100, 103, 136};

    bool checkRef(float* newC){
      for (int i = 0; i < M*N; i++) {
        if (newC[i] != C[i]) {
          std::cout << "fail at: " << i << ", values: " << newC[i] << " vs. " << C[i] << std::endl;
          return false;
        }
      }
      return true;
    }
};

#endif //UTILS_H


Writing /home/include/utils.h


In [14]:
%%writefile my_gpu.cu
#include "/home/include/utils.h"
#include <cuda_runtime.h>

#define NUM_RUNS 10

#define CUDA_CHECK(func)                                                     	   \
    do {                                                                           \
        cudaError_t status = (func);                                               \
        if (status != cudaSuccess) {                                               \
            printf("CUDA API failed at line %d with error: %s (%d)\n", __LINE__,   \
                cudaGetErrorString(status), status);                               \
            exit(EXIT_FAILURE);                                                    \
        }                                                                          \
    } while (0)

#define CHECK(name) \
    float *d_Aref_ ## name, *d_Bref_ ## name, *d_Cref_ ## name; \
    std::cerr << "checking " << #name << std::endl; \
    CUDA_CHECK(cudaMalloc(&d_Aref_ ## name, Ref::M * Ref::K * sizeof(float))); \
    CUDA_CHECK(cudaMalloc(&d_Bref_ ## name, Ref::K * Ref::N * sizeof(float))); \
    CUDA_CHECK(cudaMalloc(&d_Cref_ ## name, Ref::M * Ref::N * sizeof(float))); \
    CUDA_CHECK(cudaMemcpy(d_Aref_ ## name, ref.A, Ref::M * Ref::K * sizeof(float), cudaMemcpyHostToDevice)); \
    CUDA_CHECK(cudaMemcpy(d_Bref_ ## name, ref.B, Ref::K * Ref::N * sizeof(float), cudaMemcpyHostToDevice)); \
    float* d_Cref_INI_ ## name = new float[M * N](); \
    for (int i = 0; i < Ref::M; i++) { \
        for (int j = 0; j < Ref::N; j++) { \
            d_Cref_INI_ ## name[i * Ref::N + j] = 0; \
        } \
    } \
    CUDA_CHECK(cudaMemcpy(d_Cref_ ## name, d_Cref_INI_ ## name, Ref::M * Ref::N * sizeof(float), cudaMemcpyHostToDevice)); \
    name(d_Aref_ ## name, d_Bref_ ## name, d_Cref_ ## name, Ref::M, Ref::N, Ref::K); \
    cudaError_t err_c_ ## name = cudaGetLastError(); \
    if (err_c_ ## name != cudaSuccess) { \
        std::cerr << "CUDA Error: " << cudaGetErrorString(err_c_ ## name) << std::endl; \
    } \
    CUDA_CHECK(cudaMemcpy(refC, d_Cref_ ## name, Ref::M * Ref::N * sizeof(float), cudaMemcpyDeviceToHost)); \
    if (!ref.checkRef(refC)){ \
        std::cerr << "check ref failed!" << std::endl; \
    };

#define TIME(name) \
    float *d_A_ ## name, *d_B_ ## name, *d_C_ ## name; \
    CUDA_CHECK(cudaMalloc(&d_A_ ## name, M * K * sizeof(float))); \
    CUDA_CHECK(cudaMalloc(&d_B_ ## name, K * N * sizeof(float))); \
    CUDA_CHECK(cudaMalloc(&d_C_ ## name, M * N * sizeof(float))); \
    CUDA_CHECK(cudaMemcpy(d_A_ ## name, A, M * K * sizeof(float), cudaMemcpyHostToDevice)); \
    CUDA_CHECK(cudaMemcpy(d_B_ ## name, B, K * N * sizeof(float), cudaMemcpyHostToDevice)); \
    cudaEvent_t start_ ## name, end_ ## name; \
    cudaEventCreate(&start_ ## name); \
    cudaEventCreate(&end_ ## name); \
    float* d_C_INI_ ## name = new float[M * N](); \
    for (int i = 0; i < M; i++) { \
        for (int j = 0; j < N; j++) { \
            d_C_INI_ ## name[i * N + j] = 0; \
        } \
    } \
    for (int i = 0; i < 2; i++) \
    { \
        CUDA_CHECK(cudaMemcpy(d_C_ ## name, d_C_INI_ ## name, M * N * sizeof(float), cudaMemcpyHostToDevice)); \
        name(d_A_ ## name, d_B_ ## name, d_C_ ## name, M, N, K); \
    } \
    cudaError_t err_t_ ## name = cudaGetLastError(); \
    if (err_t_ ## name != cudaSuccess) { \
        std::cerr << "CUDA Error: " << cudaGetErrorString(err_t_ ## name) << std::endl; \
    } \
    float milliseconds_ ## name = 0; \
    for (int i = 0; i < NUM_RUNS; i++) \
    { \
        CUDA_CHECK(cudaMemcpy(d_C_ ## name, d_C_INI_ ## name, M * N * sizeof(float), cudaMemcpyHostToDevice)); \
        cudaDeviceSynchronize(); \
        cudaEventRecord(start_ ## name); \
        name(d_A_ ## name, d_B_ ## name, d_C_ ## name, M, N, K); \
        cudaEventRecord(end_ ## name); \
        cudaEventSynchronize(end_ ## name); \
        float milliseconds_ ## i = 0; \
        cudaEventElapsedTime(&milliseconds_ ## i, start_ ## name, end_ ## name); \
        milliseconds_ ## name += milliseconds_ ## i; \
    } \
    cudaMemcpy(C, d_C_ ## name, M * N * sizeof(float), cudaMemcpyDeviceToHost); \
    std::cout << "Time taken for GEMM (GPU, " << #name <<"): " << milliseconds_ ## name / (float)NUM_RUNS << "ms" << std::endl; \
    cudaFree(d_A_ ## name); \
    cudaFree(d_B_ ## name); \
    cudaFree(d_C_ ## name);

__global__ void gemm_gpu_o0_kernel(float* A, float* B, float *C, int M, int N, int K) {
    if (threadIdx.x == 0 && blockIdx.x == 0) {
        for (int i = 0; i < M; i++) {
            for (int j = 0; j < N; j++) {
                for (int k = 0; k < K; k++) {
                    C[i * N + j]  += A[i * K + k]  * B[k * N + j];
                }
            }
        }
    }
}

void gemm_gpu_o0(float* A, float* B, float* C, int M, int N, int K)
{
    // Init block and grid size
    dim3 blockSize(1);
    dim3 gridSize(1);
    gemm_gpu_o0_kernel<<<gridSize, blockSize>>>(A, B, C, M, N, K);
}

// The scafolding for optimized GEMM implementations
__global__ void gemm_gpu_o1_kernel(float* A, float* B, float *C, int M, int N, int K) {
    // M x K * K x N = M x N
    float Cvalue = 0;
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    if ((row < M) && (col < N)) {
        for (int e = 0; e < K; e++) {
        Cvalue += A[row * K + e] * B[e * N + col];
        }
        C[row * N + col] = Cvalue;
    }
}
void gemm_gpu_o1(float* A, float* B, float* C, int M, int N, int K)
{
    // Init block and grid size
    dim3 blockSize(16, 16);
    dim3 gridSize((N + blockSize.x - 1) / blockSize.x, (M + blockSize.y - 1) / blockSize.y);
    gemm_gpu_o1_kernel<<<gridSize, blockSize>>>(A, B, C, M, N, K);
}

#define TILE_WIDTH_O2 8
__global__ void gemm_gpu_o2_kernel(float* A, float* B, float *C, int M, int N, int K) {
    __shared__ float subTileA[TILE_WIDTH_O2][TILE_WIDTH_O2];
    __shared__ float subTileB[TILE_WIDTH_O2][TILE_WIDTH_O2];

    int bx = blockIdx.x;
    int by = blockIdx.y;
    int tx = threadIdx.x;
    int ty = threadIdx.y;
    int row = by * TILE_WIDTH_O2 + ty;
    int col = bx * TILE_WIDTH_O2 + tx;
    float Pvalue = 0;
    for (int q = 0; q < ceil(1.0 * K / TILE_WIDTH_O2); q++) {
        if ((row < M) && (q * TILE_WIDTH_O2 + tx < K))
            subTileA[ty][tx] = A[row * K + q * TILE_WIDTH_O2 + tx];
        else
            subTileA[ty][tx] = 0;
        if ((col < N) && (q * TILE_WIDTH_O2 + ty < K))
            subTileB[ty][tx] = B[(q * TILE_WIDTH_O2 + ty) * N + col];
        else
            subTileB[ty][tx] = 0;
        __syncthreads();
        for (int k = 0; k < TILE_WIDTH_O2; k++) {
            Pvalue += subTileA[ty][k] * subTileB[k][tx];
        }
        __syncthreads();
    }
    if ((row < M) && (col < N))
        C[row * N + col] = Pvalue;
}
void gemm_gpu_o2(float* A, float* B, float* C, int M, int N, int K)
{
    // Init block and grid size
    dim3 blockSize(TILE_WIDTH_O2, TILE_WIDTH_O2);
    dim3 gridSize(ceil((1.0 * N) / TILE_WIDTH_O2), ceil((1.0 * M) / TILE_WIDTH_O2));
    gemm_gpu_o2_kernel<<<gridSize, blockSize>>>(A, B, C, M, N, K);
}

#define TILE_WIDTH_O3 32
__global__ void gemm_gpu_o3_kernel(float* A, float* B, float *C, int M, int N, int K) {
    __shared__ float subTileA[TILE_WIDTH_O3][TILE_WIDTH_O3];
    __shared__ float subTileB[TILE_WIDTH_O3][TILE_WIDTH_O3];

    int bx = blockIdx.x;
    int by = blockIdx.y;
    int tx = threadIdx.x;
    int ty = threadIdx.y;
    int row = by * TILE_WIDTH_O3 + ty;
    int col = bx * TILE_WIDTH_O3 + tx;
    float Pvalue = 0;
    for (int q = 0; q < ceil(1.0 * K / TILE_WIDTH_O3); q++) {
        if ((row < M) && (q * TILE_WIDTH_O3 + tx < K))
            subTileA[ty][tx] = A[row * K + q * TILE_WIDTH_O3 + tx];
        else
            subTileA[ty][tx] = 0;
        if ((col < N) && (q * TILE_WIDTH_O3 + ty < K))
            subTileB[ty][tx] = B[(q * TILE_WIDTH_O3 + ty) * N + col];
        else
            subTileB[ty][tx] = 0;
        __syncthreads();
        for (int k = 0; k < TILE_WIDTH_O3; k++) {
            Pvalue += subTileA[ty][k] * subTileB[k][tx];
        }
        __syncthreads();
    }
    if ((row < M) && (col < N))
        C[row * N + col] = Pvalue;
}
void gemm_gpu_o3(float* A, float* B, float* C, int M, int N, int K)
{
    // Init block and grid size
    dim3 blockSize(TILE_WIDTH_O3, TILE_WIDTH_O3);
    dim3 gridSize(ceil((1.0 * N) / TILE_WIDTH_O3), ceil((1.0 * M) / TILE_WIDTH_O3));
    gemm_gpu_o3_kernel<<<gridSize, blockSize>>>(A, B, C, M, N, K);
}

#include "cublas_v2.h"
void gemm_cublas(float* A, float* B, float* C, int M, int N, int K)
{
    // ref: https://docs.nvidia.com/cuda/cublas/#cublas-t-gemm

    cublasHandle_t handle;
    cublasCreate(&handle);
    const float alpha = 1.0f;
    const float beta = 0.0f;
    cublasSgemm(handle, CUBLAS_OP_N, CUBLAS_OP_N, N, M, K, &alpha, B, N, A, K, &beta, C, N);
    cublasDestroy(handle);
}

int main(int argc, char* argv[]) {
    if (argc < 3) {
        std::cout << "Usage: mp1 <M> <N> <K>" << std::endl;
        return 1;
    }

    int M = atoi(argv[1]);
    int N = atoi(argv[2]);
    int K = atoi(argv[3]);

    // int runs = atoi(argv[3]);
    float* A = new float[M * K]();
    float* B = new float[K * N]();
    float* C = new float[M * N]();

    fillRandom(A, M * K);
    fillRandom(B, K * N);

    /// GPU Implementation
        // Check if implementation is correct
    auto ref = Ref();
    float* refC = new float[Ref::M * Ref::N]();
    // CHECK(gemm_gpu_o0)
    CHECK(gemm_gpu_o1)
    CHECK(gemm_gpu_o2)
    CHECK(gemm_gpu_o3)
    CHECK(gemm_cublas)

    // Actual run
    // TIME(gemm_gpu_o0)
    TIME(gemm_gpu_o1)
    TIME(gemm_gpu_o2)
    TIME(gemm_gpu_o3)
    TIME(gemm_cublas)

    cudaFreeHost(A);
    cudaFreeHost(B);
    cudaFreeHost(C);

    delete[] A;
    delete[] B;
    delete[] C;

    return 0;
}

Overwriting my_gpu.cu


In [17]:
!nvcc my_gpu.cu -o my_gpu -lcublas
!./my_gpu 10000 10000 10000

checking gemm_gpu_o1
checking gemm_gpu_o2
checking gemm_gpu_o3
checking gemm_cublas
Time taken for GEMM (GPU, gemm_gpu_o1): 5709.66ms
Time taken for GEMM (GPU, gemm_gpu_o2): 5929.47ms
Time taken for GEMM (GPU, gemm_gpu_o3): 2551.28ms
Time taken for GEMM (GPU, gemm_cublas): 534.726ms
